# Lab 4: Mode Choice

## Estimation dataset

We are using the same 2017 survey trip records for estimation. However, the data has been pre-processed for you to allow for model estimation. The file **'trip17_estimation.csv'** contains all of the observed rows, but also has additional rows for the unchosen alternative by mode. To measure and estimate the relative value of one mode alternative to the others, we need data on those other alternatives, which generally aren't provided in a standard survey. 

This dataset was generated by duplicating all observed (chosen) trips (total length x) for each mode (m) alternative (once for walk alternatives, once for bike, etc.), to end up with a new dataset that was of length (x)(m). Existing PSRC scripts were used to add travel time, cost, and distance for these trip alternatives, using model outputs. Model outputs report time/cost/distance between all origins and destinations (often called 'skims'), so it's possible to provided measures for all trips, even those that are highly unlikely (like biking from Tacoma to Everett). Some modellers have used other means of attaching these "skim" values to alternatives, including Google APIs. There is usually a cost associated with this since API calls are capped, and model skims are readily available, so it was easier to use the old method. However, with more observed data in the world, there are plenty of alternatives becoming available!

----

## Data dictionary
### Modes:
- 1: Walk
- 2: Bike
- 3: SOV (single occupant vehicle)
- 4: HOV2 (2 people in a vehicle)
- 5: HOV3 (3 or more people in a vehicle)
- 6: Transit (including bus and train)
- 9: TNC (hired rideshare vehicles like Uber, Lyft)

### Sociodemographics and Land Use:
Same as past codebook unless otherwise noted
- choice: 0/1 for if record represents observed choice data
- hh_density_o: households/ft^2 in trip's origin TAZ
- emp_density_o: total employees/ft^2 in trip's origin TAZ
- college_density_o: college students/ft^2 in trip's origin TAZ
- gradeschool_density_o: gradeschool students/ft^2 in trip's origin TAZ
- dist_lbus: distance to transit in miles

## Pylogit

There are multiple libraries that can be used to estimate choice models in Python, including:
- [statsmodels](https://www.statsmodels.org/stable/index.html)
- [pylogit](https://github.com/timothyb0912/pylogit)
- [choicemodels (built partially off pylogit)](https://github.com/UDST/choicemodels)
- [biogeme](http://biogeme.epfl.ch/examples_swissmetro.html)

We will focus on pylogit because it is easier and more flexible to use than statsmodels, and more developed than choicemodels, which is still in development. Biogeme is an old tool that is now available in pandas, which is very nice, but the documentation is somewhat lacking. You are welcome to try multiple libraries, but I will only be able to provide direct supply for pylogit.

For more information, see: https://github.com/timothyb0912/pylogit/tree/master/examples/notebooks. 
- You can open HTML notebooks from these links, e.g. https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Main%20PyLogit%20Example.ipynb
- If interested in nested logit, see example: https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Nested%20Logit%20Example--Python%20Biogeme%20benchmark--09NestedLogit.ipynb

In [1]:
import pandas as pd
import numpy as np

# Import the pylogit library
import pylogit as pl    # Importing as shortcut "pl", similar to pandas imported as "pd"
from collections import OrderedDict    # a requirement for pylogit specifications

In [2]:
# Load a list of HBW trips in estimation format

df_hbw = pd.read_csv(r'data/trip17_estimation_hbw.csv')

In [3]:
# what does the trips dataset look like now?
df_hbw

,hhno,pno,mode,opurp,dpurp,deptm,otaz,dtaz,arrtm,trexpfac,...,dist_lbus,hhid,hhsize,vehicle_count,numchildren,numworkers,hhincome_broad,rent_own,res_type,intercept
0,17100248,1,1,1,0,0,327,429,0,119.663798,...,0.069752,17100248,1,0,0,1,15.0,1,4,1.0
1,17100248,1,2,1,0,0,327,429,0,119.663798,...,0.069752,17100248,1,0,0,1,15.0,1,4,1.0
2,17100248,1,3,1,0,0,327,429,0,119.663798,...,0.069752,17100248,1,0,0,1,15.0,1,4,1.0
3,17100248,1,4,1,0,0,327,429,0,119.663798,...,0.069752,17100248,1,0,0,1,15.0,1,4,1.0
4,17100248,1,5,1,0,0,327,429,0,119.663798,...,0.069752,17100248,1,0,0,1,15.0,1,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34526,17148237,1,3,1,0,0,907,778,0,0.000000,...,0.104494,17148237,1,1,0,1,17.5,2,3,1.0
34527,17148237,1,4,1,0,0,907,778,0,0.000000,...,0.104494,17148237,1,1,0,1,17.5,2,3,1.0
34528,17148237,1,5,1,0,0,907,778,0,0.000000,...,0.104494,17148237,1,1,0,1,17.5,2,3,1.0
34529,17148237,1,6,1,0,0,907,778,0,0.000000,...,0.104494,17148237,1,1,0,1,17.5,2,3,1.0


## MNL estimation specification

In [4]:
# We are using what is called an Ordered Dictionary
# Remember that a dictionary looks like this {'key': 'value'}
# An ordered dictionary is a special version of this type that keeps the keys in order

specification = OrderedDict()
names = OrderedDict()

In [5]:
# Define the alternative specific constants (ASCs), i.e., the intercepts
# Remember that one choice is the baseline (ASC=0)
# Leave the chosen baseline mode out of the list of intercept values below
# In this case, we are using SOV as the base model - it's common to use the most likely alternative as the base
specification["intercept"] = [1, 2, 4, 5, 6, 9]    # these are the mode IDs, excluding 3 for SOV
names['intercept'] = ['Walk ASC','Bike ASC', 'HOV2 ASC','HOV3+ ASC','Transit ASC', 'TNC']    # Provide labels

specification

OrderedDict([('intercept', [1, 2, 4, 5, 6, 9])])

In [6]:
# Create a coefficient for travel time
# Note that we are using only a single travel time coefficient across all modes
# Lists of lists denote alternatives will share a common coefficient

specification['travtime'] = [[1,2,3,4,5,6,9]]    # Note that this is a list inside a list [[]]
names['travtime'] = ['time all']

names

OrderedDict([('intercept',
              ['Walk ASC',
               'Bike ASC',
               'HOV2 ASC',
               'HOV3+ ASC',
               'Transit ASC',
               'TNC']),
             ('travtime', ['time all'])])

In [7]:
# Specify which columns are used in the estimation
custom_alt_id = "mode_id"    # Mode columns, must be integer based
obs_id_column = "custom_id"    # an ID that is unique to each choice (a set of chosen and unchosen alternatives have their own ID)
choice_column = "choice"    # 0/1 column indicating if that row was the chosen or unchosen alternative

# Call the module to create the choice model specification
model_1 = pl.create_choice_model(data=df_hbw,    # Note that here's we are specifying the df_hbw dataset
                                alt_id_col=custom_alt_id,
                                obs_id_col=obs_id_column,
                                choice_col=choice_column,
                                specification=specification,    # using the basic_specification from above
                                model_type="MNL",
                                names=names)

In [8]:
model_1

In [9]:
# The code above only generated the template to estimate the model. We still need to execute the actual
# maximum likelihood estimation process. 

# Run the estimation given the specification "basic_mnl"

# Specify the initial values and method for the optimization.
# Note that the value in np.zeros() is the number of coefficients we expect to return, including ASCs
# For the basic setup above there are 6 ASC alternatives and 1 travel time variable, for a total of 7
# Note that the error result if you get this number wrong will usually give the required value. 

# Note that here's were using a method called "fit_mile" on the object "basic_mnl," which we created above. 
model_1.fit_mle(np.zeros(7))
print(np.round(model_1.summary, 3))    # Make things easier to read

# Look at the estimation results
print(model_1.get_statsmodels_summary())

Log-likelihood at zero: -9,595.2829
Initial Log-likelihood: -9,595.2829


/Users/kaitlynng/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py:527: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.53 seconds.
Final log-likelihood: -6,436.4076
             parameters  std_err  t_stats  p_values  robust_std_err  \
Walk ASC          0.705    0.074    9.493       0.0           0.127   
Bike ASC         -1.554    0.073  -21.240       0.0           0.084   
HOV2 ASC         -1.735    0.054  -31.944       0.0           0.054   
HOV3+ ASC        -3.433    0.119  -28.871       0.0           0.119   
Transit ASC      -0.355    0.036   -9.839       0.0           0.039   
TNC              -3.548    0.126  -28.204       0.0           0.126   
time all         -0.038    0.002  -23.033       0.0           0.004   

             robust_t_stats  robust_p_values  
Walk ASC              5.565              0.0  
Bike ASC            -18.402              0.0  
HOV2 ASC            -31.942              0.0  
HOV3+ ASC           -28.870              0.0  
Transit ASC          -9.093              0.0  
TNC                 -28.204              0.0  
time all        

# Comparing to a new model

## Hypothesis test

State the null $H_0$ and the alternative hypothesis $H_a$:
- $H_0: \beta(travel time) = \beta(travel time_x)$ for all of $x$ modes - Model 1
- $H_a: \beta(travel time) \neq \beta(travel time_x)$ for all of $x$ modes - Model 2

We will reject the null hypothesis ($H_0$) if there is a statistically significant difference between the two models based on the likelihood ratio test.

Intuitively, this test is saying that we think there might be differences in the way time is valued across modes. The null is saying that all modes users have a shared time valuation.

## Utility equations for Model 2
Note that SOV is our base case.
- $V(walk) = ASC_{walk} + \beta(time_{walk}) * TTime_{walk}$
- $V(bike) = ASC_{bike} + \beta(time_{bike}) * TTime_{bike}$
- $V(hov2) = ASC_{hov2} + \beta(time_{hov2}) * TTime_{hov2}$
- $V(hov3) = ASC_{hov3} + \beta(time_{hov3}) * TTime_{hov3}$
- $V(transit) = ASC_{transit} + \beta(time_{transit}) * TTime_{transit}$
- $V(tnc) = ASC_{tnc} + \beta(time_{tnc}) * TTime_{tnc}$
- $V(sov) = \beta(time_{sov}) * TTime_{sov}$

In [10]:
# We are using what is called an Ordered Dictionary
# Remember that a dictionary looks like this {'key': 'value'}
# An ordered dictionary is a special version of this type that keeps the keys in order

specification = OrderedDict()
names = OrderedDict()

# Define the alternative specific constants (ASCs), i.e., the intercepts
# Remember that one choice is the baseline (ASC=0)
# Leave the chosen baseline mode out of the list of intercept values below
# In this case, we are using SOV as the base model - it's common to use the most likely alternative as the base
specification["intercept"] = [1, 2, 4, 5, 6, 9]    # these are the mode IDs, excluding 3 for SOV
names['intercept'] = ['Walk ASC','Bike ASC', 'HOV2 ASC','HOV3+ ASC','Transit ASC', 'TNC']    # Provide labels

# Adding a travel time coefficient for all modes
specification['travtime'] = [1,2,3,4,5,6,9]    
names['travtime'] = ['time walk','time bike','time sov','time hov2','time hov3', 'time transit','time tnc']

# Specify which columns are used in the estimation
custom_alt_id = "mode_id"    # Mode columns, must be integer based
obs_id_column = "custom_id"    # an ID that is unique to each choice (a set of chosen and unchosen alternatives have their own ID)
choice_column = "choice"    # 0/1 column indicating if that row was the chosen or unchosen alternative

# Call the module to create the choice model specification
model_2 = pl.create_choice_model(data=df_hbw,    # Note that here's we are specifying the df_hbw dataset
                                alt_id_col=custom_alt_id,
                                obs_id_col=obs_id_column,
                                choice_col=choice_column,
                                specification=specification,    # using the basic_specification from above
                                model_type="MNL",
                                names=names)

In [11]:
# In this case, there are now 13 coefficients we are estimated. Count the values in the "names" list, including ASCs and other coefficents
model_2.fit_mle(np.zeros(13))
print(np.round(model_2.summary, 3))   # Make things easier to read

# Look at the estimation results
print(model_2.get_statsmodels_summary())

Log-likelihood at zero: -9,595.2829
Initial Log-likelihood: -9,595.2829


/Users/kaitlynng/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py:527: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.53 seconds.
Final log-likelihood: -6,250.2121
              parameters  std_err  t_stats  p_values  robust_std_err  \
Walk ASC          -0.305    0.113   -2.687     0.007           0.174   
Bike ASC          -2.688    0.137  -19.578     0.000           0.153   
HOV2 ASC          -0.585    0.132   -4.428     0.000           0.147   
HOV3+ ASC         -2.767    0.283   -9.795     0.000           0.319   
Transit ASC       -1.311    0.093  -14.071     0.000           0.126   
TNC               -2.380    0.302   -7.870     0.000           0.236   
time walk         -0.045    0.002  -24.666     0.000           0.005   
time bike         -0.061    0.004  -15.101     0.000           0.005   
time sov          -0.167    0.010  -17.524     0.000           0.015   
time hov2         -0.240    0.012  -19.984     0.000           0.016   
time hov3         -0.207    0.019  -11.094     0.000           0.023   
time transit      -0.095    0.005  -19.704     0.0

# Likelihood ratio test
The unrestricted model is the one defined above (Model 2). We are not constraining $\beta(time)$ to be equal across all modes as in the first model. Generally the unrestricted model is the one with more coefficients.

The restricted model for comparison is Model 1. We want to perform the likelihood ratio test to see if there is a statistically significant difference between the specifications. If so, we can reject the null hypothesis and accept the alternative.

In [12]:
# can look at fit statistics using this command
model_1.fit_summary

Number of Parameters                                                      7
Number of Observations                                                 4933
Null Log-Likelihood                                            -9595.282945
Fitted Log-Likelihood                                           -6436.40759
Rho-Squared                                                        0.329211
Rho-Bar-Squared                                                    0.328482
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object

In [13]:
LL_restricted =  model_1.fit_summary[3]     # -6436.408, Model 1
LL_unrestricted = model_2.fit_summary[3]    # -6250.212, Model 2

-2*(LL_restricted-LL_unrestricted)    # See slides posted for this formula or the link to Koppelman and Bhat


372.39089121257894

The ratio test returns a value of 372.39. If this value is creater than the chi-square critical value, we can reject the null hypothesis. The degrees of freedom to consider for the chi-square distribution is the difference in coefficients between the two models, or 6 in this example (13 - 7). This value is far higher than the critical chi-square value at a 99.9% confidence level, so we can say that travel time coefficients provide a better measure of utility when defined for all modes separately. Given the large t-statistic for all coefficients (and intuitive signs), we will keep all variables and use this model going forward.

Chi-square table: https://faculty.washington.edu/heagerty/Books/Biostatistics/TABLES/ChiSquare/index.html

# Grouping time coefficients
An alternative to above is grouping time coefficients across somewhat similar modes. We will group non-motorized modes together ($\beta(time_{nmt})$), as well as auto modes ($\beta(time_{auto})$). Transit and TNC will have their own time coefficients. The utility formulation for this is:
- $V(walk) = ASC_{walk} + \beta(time_{nmt}) * TTime_{walk}$
- $V(bike) = ASC_{bike} + \beta(time_{nmt}) * TTime_{bike}$
- $V(hov2) = ASC_{hov2} + \beta(time_{auto}) * TTime_{hov2}$
- $V(hov3) = ASC_{hov3} + \beta(time_{auto}) * TTime_{hov3}$
- $V(transit) = ASC_{transit} + \beta(time_{transit}) * TTime_{transit}$
- $V(tnc) = ASC_{tnc} + \beta(time_{tnc}) * TTime_{tnc}$
- $V(sov) = \beta(time_{auto}) * TTime_{sov}$

In [14]:
# We are using what is called an Ordered Dictionary
# Remember that a dictionary looks like this {'key': 'value'}
# An ordered dictionary is a special version of this type that keeps the keys in order

specification = OrderedDict()
names = OrderedDict()

# Define the alternative specific constants (ASCs), i.e., the intercepts
# Remember that one choice is the baseline (ASC=0)
# Leave the chosen baseline mode out of the list of intercept values below
# In this case, we are using SOV as the base model - it's common to use the most likely alternative as the base
specification["intercept"] = [1, 2, 4, 5, 6, 9]    # these are the mode IDs, excluding 3 for SOV
names['intercept'] = ['Walk ASC','Bike ASC', 'HOV2 ASC','HOV3+ ASC','Transit ASC', 'TNC']    # Provide labels

# we will have 4 separate groups. If a group has multiple values it is placed in its own list
specification['travtime'] = [[1,2],[3,4,5],6,9]    # Note that this is a list inside a list [[]]
names['travtime'] = ['time walk and bike','time auto','time transit','time tnc']

# Specify which columns are used in the estimation
custom_alt_id = "mode_id"    # Mode columns, must be integer based
obs_id_column = "custom_id"    # an ID that is unique to each choice (a set of chosen and unchosen alternatives have their own ID)
choice_column = "choice"    # 0/1 column indicating if that row was the chosen or unchosen alternative

# Call the module to create the choice model specification
model_1 = pl.create_choice_model(data=df_hbw,    # Note that here's we are specifying the df_hbw dataset
                                alt_id_col=custom_alt_id,
                                obs_id_col=obs_id_column,
                                choice_col=choice_column,
                                specification=specification,    # using the basic_specification from above
                                model_type="MNL",
                                names=names)

In [15]:
# Note we now have 10 coefficients to be estimated
model_1.fit_mle(np.zeros(10))
print(np.round(model_1.summary, 3))    # Make things easier to read

# Look at the estimation results
print(model_1.get_statsmodels_summary())

Log-likelihood at zero: -9,595.2829
Initial Log-likelihood: -9,595.2829


/Users/kaitlynng/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py:527: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.46 seconds.
Final log-likelihood: -6,306.7316
                    parameters  std_err  t_stats  p_values  robust_std_err  \
Walk ASC                -0.274    0.097   -2.836     0.005           0.123   
Bike ASC                -3.043    0.128  -23.773     0.000           0.151   
HOV2 ASC                -1.738    0.054  -32.001     0.000           0.054   
HOV3+ ASC               -3.437    0.119  -28.900     0.000           0.119   
Transit ASC             -1.384    0.087  -15.935     0.000           0.118   
TNC                     -2.559    0.301   -8.513     0.000           0.232   
time walk and bike      -0.047    0.002  -25.799     0.000           0.005   
time auto               -0.162    0.009  -18.784     0.000           0.014   
time transit            -0.089    0.004  -19.951     0.000           0.007   
time tnc                -0.226    0.021  -10.769     0.000           0.019   

                    robust_t_stats  robust_p_values  
W

In [16]:
# !jupyter nbconvert --to html "Lab 4 In Class.ipynb"